### Dynamic Prompting

This example goes over how to use DynamicPrompting to interact with dynamic few-shot prompting inference for downstream tasks such as classification.



In [ ]:
!pip install dynamic_prompting

In [ ]:
!git clone https://github.com/ElmiraGhorbani/dynamic_prompting

#### Import the necessary packages

In [ ]:
import pandas as pd
from dynamic_prompting.embedings.config import EmbeddingsConfig
from dynamic_prompting.knowledge_base.config import KnowledgeBaseConfig
from dynamic_prompting.knowledge_base.knowledge_base import \
    KnowledgeBaseManagement
from dynamic_prompting.llms.config import LLMConfig, PromptConfig
from dynamic_prompting.llms.llm import LlamaModel
from dynamic_prompting.llms.prompt import PromptManagement
from dynamic_prompting.utils.utils import get_project_root

#### Set configs

##### Embedding Model

In [ ]:
# Select one of the following options, just uncomment one.

# Nomic: https://huggingface.co/nomic-ai/nomic-embed-text-v1.5
emb_config = EmbeddingsConfig(
    model_name="nomic-embed-text-v1.5",
    local_files=True,
    trust_remote_code=True,
    max_dimension=512,
)


# Mxbai: https://huggingface.co/mixedbread-ai/mxbai-embed-large-v1

# emb_config = EmbeddingsConfig(
#     model_name="mxbai-embed-large-v1",
#     local_files=True,
#     trust_remote_code=False,
#     max_dimension=512,
# )


# BGE : https://huggingface.co/BAAI/bge-small-en-v1.5
# emb_config = EmbeddingsConfig(
#     model_name="bge-small-en-v1.5",
#     local_files=True,
# )

##### Knowledge Base Config

In [ ]:
kb_config = KnowledgeBaseConfig(
    knowledge_base_name='nomic_text_classification')

##### LLM Config

In [ ]:
llm_config = LLMConfig(
    rank=0,
    world_size=1,
    max_seq_len=1024,
    max_batch_size=8,
    local_files=True
)

#### Prompt Config

In [ ]:
# This example is for the classification case

prompt = """
### Instruction:
You are given a text sample, and your task is to classify it into one of the following categories: Business, Science, Sports,\n
Politics, Entertainment, Weather, Technology, Health, Local, World, Culture, Education, Travel.\n
Carefully read the text and determine the most appropriate category that best describes the main topic of the text.\n

Examples:
{examples}

**Text:** "Your text sample here"
**Category:** "Only return the category"
"""
prompt_config = PromptConfig(prompt=prompt)

In [ ]:
kb = KnowledgeBaseManagement(embeddings_config=emb_config, kb_config=kb_config)

llama_model = LlamaModel(llm_config=llm_config)


prompt_handler = PromptManagement(prompt_config=prompt_config)

root_path = get_project_root()

#### Dataset

Check the data folder to learn about the data structure for dynamic few-shot prompting.

In [ ]:
df = pd.read_csv(
    f'{root_path}/data/sample.csv',
    index_col=False
)
messages = df['txt'].tolist()
messages = [i.replace('\n', ' ') for i in messages]
labels = df['label'].tolist()

#### Create the knowledgebase

In [ ]:
kb.create_kb(messages)

db = kb.load_kb()

#### Run Dynamic inference

In [ ]:
query = '''The city council has announced a new initiative to improve public transportation\
    infrastructure, aiming to reduce traffic congestion and promote eco-friendly travel options.'''

indx = kb.search_kb(
    query=query,
    embeddings=db,
    num_of_neighbours=5
)


samples = [messages[i] for i in indx]
labels = [labels[i] for i in indx]

prompt = prompt_handler.set_few_shots(context_input=samples, labels=labels)

# run model prediction
inf_result = llama_model.inference(
    system_instruction=prompt,
    user_input=query
)

print(inf_result)